# Extract RXCUI relationships from the `RXNREL.RRF` file

2019-04-22

The RxNorm semantic network is contained in the `RXNREL.RRF` file.
We will extract the RXCUI relationships so that we can traverse the network in order to determine the active ingredients of each drug.

In [1]:
import pandas as pd

## Read `RXNREL.RRF`

In [2]:
rawd = (pd
    .read_csv(
        "../../data/rxnorm/RXNREL.RRF", sep="|", names=[
            "rxcui1", "rxaui1", "stype1", "rel",
            "rxcui2", "rxaui2", "stype2", "rela",
            "rui", "srui", "sab", "sl", "dir",
            "rg", "suppress", "cvf",
            "temp"
        ]
    )
    .dropna(how="all", axis=1)
)

In [3]:
rawd.shape

(6325984, 12)

In [4]:
rawd.head()

,rxcui1,rxaui1,stype1,rel,rxcui2,rxaui2,stype2,rela,rui,sab,dir,cvf
0,NaN,5.0,AUI,SY,NaN,6.0,AUI,permuted_term_of,131943934.0,MSH,NaN,NaN
1,NaN,5.0,SDUI,SIB,NaN,104746.0,SDUI,NaN,132692151.0,MSH,NaN,NaN
2,NaN,5.0,SDUI,RN,NaN,609702.0,SDUI,mapped_to,131849389.0,MSH,1.0,NaN
3,NaN,5.0,AUI,SY,NaN,2666961.0,AUI,sort_version_of,132861204.0,MSH,NaN,NaN
4,NaN,5.0,AUI,SY,NaN,2681015.0,AUI,entry_version_of,131939748.0,MSH,NaN,NaN


### Empty cells

In [5]:
(rawd
    .isnull()
    .sum()
    .to_frame("num_null")
    .assign(pct_null = lambda df: df["num_null"].divide(len(rawd)).multiply(100))
)

,num_null,pct_null
rxcui1,4833664,76.409678
rxaui1,1492320,23.590322
stype1,0,0.000000
rel,0,0.000000
rxcui2,4833664,76.409678
rxaui2,1492320,23.590322
stype2,0,0.000000
rela,521334,8.241153
rui,497240,7.860279
sab,0,0.000000


The RXCUI columns have more nulls.
There seems to be more relationships between RXAUIs.

### Column data types

In [6]:
rawd.dtypes

rxcui1    float64
rxaui1    float64
stype1     object
rel        object
rxcui2    float64
rxaui2    float64
stype2     object
rela       object
rui       float64
sab        object
dir       float64
cvf       float64
dtype: object

We will need to normalize the float columns to ints.

### Concept types

In [7]:
rawd["stype1"].value_counts()

AUI     4807210
CUI     1492320
SDUI      19252
SCUI       7202
Name: stype1, dtype: int64

In [8]:
rawd["stype2"].value_counts()

AUI     4807210
CUI     1492320
SDUI      19252
SCUI       7202
Name: stype2, dtype: int64

### Can we use the stype columns to only get the RXCUI relationships?

Check if we will miss any RXCUI relationships.

In [9]:
(rawd
    [rawd["rxcui1"].isnull()]
    ["stype1"].value_counts()
)

AUI     4807210
SDUI      19252
SCUI       7202
Name: stype1, dtype: int64

In [10]:
(rawd
    [rawd["rxcui2"].isnull()]
    ["stype1"].value_counts()
)

AUI     4807210
SDUI      19252
SCUI       7202
Name: stype1, dtype: int64

The stype columns will never contain "CUI" if we want RXCUI relationships.

In [11]:
(rawd
    [rawd["rxcui1"].notnull()]
    ["stype1"].value_counts()
)

CUI    1492320
Name: stype1, dtype: int64

In [12]:
(rawd
    [rawd["rxcui2"].notnull()]
    ["stype1"].value_counts()
)

CUI    1492320
Name: stype1, dtype: int64

The columns will only contain "CUI" if it's a RXCUI relationship.

---

## Use the stype columns to filter down to RXCUI relationships

In [13]:
cui_rels = (rawd
    .query("stype1 == 'CUI' and stype2 == 'CUI'")
    .drop(["stype1", "stype2"], axis=1)
    .dropna(how="all", axis=1)
    .assign(
        rxcui1 = lambda df: df["rxcui1"].astype("int64"),
        rxcui2 = lambda df: df["rxcui2"].astype("int64"),
        rui = lambda df: df["rui"].astype("int64")
    )
    .reset_index(drop=True)
)

In [14]:
cui_rels.shape

(1492320, 7)

In [15]:
cui_rels.head()

,rxcui1,rel,rxcui2,rela,rui,sab,cvf
0,38,RB,1760,has_tradename,4696871,RXNORM,4096.0
1,38,RO,105050,has_ingredient,4343918,RXNORM,4096.0
2,38,RO,105445,has_ingredient,4229336,RXNORM,NaN
3,38,RO,105446,has_ingredient,3798489,RXNORM,4096.0
4,38,RO,105447,has_ingredient,4423580,RXNORM,NaN


In [16]:
cui_rels.dtypes

rxcui1      int64
rel        object
rxcui2      int64
rela       object
rui         int64
sab        object
cvf       float64
dtype: object

In [17]:
cui_rels.isnull().sum()

rxcui1         0
rel            0
rxcui2         0
rela           0
rui            0
sab            0
cvf       969996
dtype: int64

In [18]:
cui_rels["sab"].value_counts()

RXNORM    1492320
Name: sab, dtype: int64

## Create final version

The source abbreviation column is not very useful since all the values are the same.
Neither is the CVF column for our purposes.

In [19]:
final = cui_rels.drop(["sab", "cvf"], axis=1)

In [20]:
final.shape

(1492320, 5)

In [21]:
final.head()

,rxcui1,rel,rxcui2,rela,rui
0,38,RB,1760,has_tradename,4696871
1,38,RO,105050,has_ingredient,4343918
2,38,RO,105445,has_ingredient,4229336
3,38,RO,105446,has_ingredient,3798489
4,38,RO,105447,has_ingredient,4423580


### Information

In [22]:
final["rel"].value_counts()

RO    839130
RB    326595
RN    326595
Name: rel, dtype: int64

In [23]:
(final["rela"]
    .value_counts()
    .to_frame("num_rels")
    .reset_index()
    .rename(columns={"index": "rela"})
    .sort_values(["num_rels", "rela"], ascending=[False, True])
    .reset_index(drop=True)
)

,rela,num_rels
0,inverse_isa,202708
1,isa,202708
2,has_ingredient,155438
3,ingredient_of,155438
4,consists_of,110911
5,constitutes,110911
6,has_tradename,106752
7,tradename_of,106752
8,dose_form_of,89850
9,has_dose_form,89850


## Only one edge per unique node pair?

In [24]:
final.groupby(["rxcui1", "rxcui2"]).size().value_counts()

1    1492320
dtype: int64

Yes, there is only one edge between each unique CUI pair.

## Relationships are reciprocal?

In [25]:
set(final["rxcui1"]) == set(final["rxcui2"])

True

## Save to file

In [26]:
final.to_csv("../../pipeline/rxnorm/rxcui_rels.tsv", sep='\t', index=False)